In [1]:
import pandas as pd 

In [2]:
#----------------------------------------------------------------------
#? 基因表达量文件跑peer时，不需要header与index
#* 对基因表达量进行进一步过滤
#? 表达量大于0.5 的样本占比超过20% 
#? 存在两倍表达差异
#----------------------------------------------------------------------
geneExpression=pd.read_csv("FPKM/gene_expression.txt",header=0,index_col=0,sep="\t")
geneExpression=geneExpression.T
lowIndex=int(geneExpression.shape[0]*0.05)
highIndex=int(geneExpression.shape[0]*0.95)
fiterGenePair=[]
for gene,ratio in geneExpression.apply(lambda x: len([i for i in x if i>0.1])/geneExpression.shape[0],axis=0).iteritems():
        if ratio>=0.05:
            lowExpression=geneExpression[gene].sort_values()[lowIndex]
            highExpression=geneExpression[gene].sort_values()[highIndex]
            #* 基因在至少20%样本中表达
            if (highExpression-lowExpression)/(highExpression+lowExpression)>=0.33:
                #* 95%和5%的样本存在表达差异
                fiterGenePair.append(gene) 

fiter_expression=geneExpression[fiterGenePair]
fiter_expression.to_csv("./peer/filter_gene_expression.txt",header=False,index=False,sep=",")
pd.Series(fiterGenePair).to_csv("peer/filter_genePair.txt",header=False,index=False,sep="\t")

In [1]:
#TODO 进行eQTL mapping
import pandas as pd
import tensorqtl
from tensorqtl import genotypeio, cis, trans
import os
import torch
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using {}'.format(device))

using cpu


In [3]:
#TODO 输入数据
plink_prefix_path ='/cotton/Liuzhenping/diaploidCotton_sQTL/Genotype/filter2_Q600_SNPs_joint_216_number_chr'
expression_bed = './phenotype/gene_expression_peer_5.bed.gz'
covariates_file = './PCA_qcovar.txt'


In [4]:
#TODO 加载数据
# load phenotypes and covariates
phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)

covariates_df = pd.read_csv(
    covariates_file, sep='\t', index_col=0, header=None)  # samples x covariates
# * 协变量文件，最终格式为 samples x covariates
covariates_df.drop([1], axis=1)
covariates_df = covariates_df.drop([1], axis=1)
covariates_df.index.name = 'id'
covariates_df.columns = ['PC1', 'PC2', 'PC3']
covariates_df = covariates_df.loc[phenotype_df.columns]

#* 基因型文件, 对应时期的样本编号
pr = genotypeio.PlinkReader(
    plink_prefix_path, select_samples=phenotype_df.columns)

# load genotypes and variants into data frames
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

Mapping files: 100%|██████████| 3/3 [00:05<00:00,  1.99s/it]


In [ ]:
# 进行cis-eQTL permutate分析
cis_df = cis.map_cis(genotype_df,
                         variant_df,
                         phenotype_df,
                         phenotype_pos_df,
                         covariates_df,
                         nperm=1000,
                         maf_threshold=0.05,
                         window=1000000,
                         seed=2022
                         )

In [ ]:
# 进行cis-eQTL nominal分析
cis.map_nominal(genotype_df, variant_df,
                             phenotype_df,
                             phenotype_pos_df,
                             prefix='nominalOut',
                             covariates_df=covariates_df,
                             maf_threshold=0.05,
                             window=1000000,
                             output_dir='.',
                             write_top=True, write_stats=True)

In [7]:
#TODO 进行trans-eQTL分析
#! 保留所有显著的位点
trans_df = trans.map_trans(genotype_df, phenotype_df, covariates_df,
                           return_sparse=True, pval_threshold=1, maf_threshold=0.05,
                           batch_size=20000)

trans-QTL mapping
  * 197 samples
  * 7 phenotypes
  * 3 covariates
  * 2143543 variants
    processing batch 108/108
    elapsed time: 0.47 min
  * 1410773 variants passed MAF >= 0.05 filtering
done.


In [10]:
trans_df=trans_df.loc[trans_df['phenotype_id']=='Garb_01G000040']

In [12]:
trans_df.to_csv("Garb_01G000040_trans_MAF.5.txt",header=True,index=False)